<a href="https://colab.research.google.com/github/yakovsushenok/RLBookProgrammingSolutions/blob/main/robot3_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I decided to write this code to check how exactly an agent behaves when the rewards for transitions in a maze are equal to $0$ and the reward for escaping the maze is $+1$.

This is to answer the following question from Sutton and Barto's book:

Exercise 3.7 Imagine that you are designing a robot to run a maze. You decide to give it a
reward of +1 for escaping from the maze and a reward of zero at all other times. The task
seems to break down naturally into episodes—the successive runs through the maze—so
you decide to treat it as an episodic task, where the goal is to maximize expected total
reward (3.7). After running the learning agent for a while, you find that it is showing
no improvement in escaping from the maze. What is going wrong? Have you e↵ectively
communicated to the agent what you want it to achieve?

First let's create a matrix which will represent our maze. The 2 corners at the top left and bottom right are the exits of the maze. If the agent steps into the exit, then the episode terminates.

In [1]:
import numpy as np

def maze(s):
  maze = np.zeros((s, s))
  maze[0,0], maze[-1,-1] = 1, 1
  return maze

Here is how a $4 \times 4$ maze looks like:

In [2]:
print(maze(4))

[[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]]


Now let's calculate the state values for the uniform policy. Note that if you're on the edge and you turn into the wall then you circle back to the same state. 

Under the uniform policy each action has a probability of $\frac{1}{4}$ to be taken. We calculate the state values under the uniform policy because it's interesting to see what will the estimates be with such rewards. The algorithm for evaluating the state values is:

First, initialise $v_0=0$, then iterate $$\forall s: v_{k+1}(s)\leftarrow E[R_{t+1}+\gamma v_k (S_{t+1})|s,\pi ]$$



In [3]:
def findCells(index, grid):
  """
  A function which will get the cells values neighboring a specific cell. 
  input:
      index: array of 2 entries i and j which indicate the cell's position. i is the row and j is the column
  ouput: 
      cells: 2 arrays. The first array has the row values of the neighbouring cells and the second array the column values. 
  Example:
   input: [1,1]
   output: [1, 0, 1, 2], [0, 1, 2, 1]
  """
  i, j = index[0], index[1]
  # Terminal state
  if (np.all(grid[i, :] == grid[0, :]) and np.all(grid[:, j] == grid[:, 0])) or (np.all(grid[i, :] == grid[-1, :]) and np.all(grid[:, j] == grid[:, -1])):
    return None, None
  # Top wall, not corner
  elif np.all(grid[i, :] == grid[0, :]) and np.all(grid[:, j] == grid[:,-1]) == False:
    return [i, i, i, i+1], [j-1, j, j+1, j]
  # Top wall, corner
  elif np.all(grid[i, :] == grid[0, :]) and np.all(grid[:, j] == grid[:,-1]):
    return [i, i, i, i+1], [j-1, j, j, j]
  # Bottom wall, not corner
  elif np.all(grid[i, :] == grid[-1, :]) and np.all(grid[:, j] == grid[:, 0])==False:
    return [i, i-1, i, i], [j-1, j, j+1, j]
  # Bottom wall, corner
  elif np.all(grid[i, :] == grid[-1, :]) and np.all(grid[:, j] == grid[:, 0]):
    return [i, i-1, i, i], [j, j, j+1, j]
  # Left wall, not corner
  elif np.all(grid[:, j] == grid[:, 0]) and np.all(grid[i,:]== grid[-1,:])==False:
    return [i, i-1, i, i+1], [j, j, j+1, j]
  # Right wall, not corner
  elif np.all(grid[:, j] == grid[:, -1]) and np.all(grid[i, :] == grid[-1,:])==False:
    return [i, i-1, i, i+1], [j-1, j, j, j]
  # No wall
  else:
    return [i, i-1, i, i+1], [j-1, j, j+1, j]
  
def robot_experiment(iterations, discount, maze_size):
  ITERATIONS = iterations
  grid = maze(maze_size)
  gridCollection = [maze(4)]
  probs = np.array([0.25, 0.25, 0.25, 0.25])
  for iter in range(ITERATIONS):
      for row in range(grid.shape[0]):
        for col in range(grid.shape[1]):
          index = [row, col]
          rowID, colID = findCells(index, grid)
          if rowID == None or colID == None:
            grid[row, col] = grid[row, col]
          else:
            last_grid = gridCollection[iter]
            grid[row, col] = discount*last_grid[rowID, colID] @ probs
      new_grid = grid.copy()
      gridCollection.append(new_grid)
      if (iter+1)% iterations==0:
        print(f"Iteration: {iter+1}, grid:")
        print(np.round(grid, 4))
        print("\n")

I am now going to see what the state values converge to. I will try values with a discount $\gamma \in \{0.1,0.9,1\}$. I will do 200 iterations as it's enough for convergence.

In [4]:
robot_experiment(200, 0.1, 4)
robot_experiment(200, 0.9, 4)
robot_experiment(200, 1, 4)

Iteration: 200, grid:
[[1.00e+00 2.57e-02 7.00e-04 0.00e+00]
 [2.57e-02 1.30e-03 1.00e-04 7.00e-04]
 [7.00e-04 1.00e-04 1.30e-03 2.57e-02]
 [0.00e+00 7.00e-04 2.57e-02 1.00e+00]]


Iteration: 200, grid:
[[1.     0.4722 0.2872 0.2349]
 [0.4722 0.3394 0.2819 0.2872]
 [0.2872 0.2819 0.3394 0.4722]
 [0.2349 0.2872 0.4722 1.    ]]


Iteration: 200, grid:
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]




The exercise assumes that this episodic task doesn't have a discount factor and that's why the robot doesn't "improve", i.e. find an optimal path to the exit. We can see this in the case where $\gamma = 1$ - there is no discount and all of the states yield the same reward which is $1$. We can see this by writing out the return at time $t$:

$$G_t = R_{t+1} + R_{t+2} +...+R_{T} = R_{T} = 1$$

So this means that at each time step, given we use the greedy policy, we would pick a random state to go to (because each state would be the maximum). This makes the agent not have a designated path towards the exit and it will just go in circles, choosing random actions, until it randomly takes an exit (goes into the terminal state).

On the other hand when $\gamma \in (0,1)$ the return at time step $t$ will be 

$$G_t = R_{t+1} + \gamma R_{t+2} + ... + \gamma^{T-t-1} R_T = R_T\gamma^{T-t-1} = (1)\gamma^{T-t-1}=\gamma^{T-t-1}$$ 

We can see that if $t$ is close to $T$, then the reward is bigger. E.g. let $\gamma = 0.9$, $T=5$ and $t_1= 2$ and $t_2=4$ ($t_1$ and $t_2$ are time steps in different episodes), then

 $$G_{t_1} = G_2 = \gamma^{5-2-1}= \gamma^{2} = 0.9^2 = 0.81 < G_{t_2} = G_4 = \gamma^{5-4-1} = \gamma^{0} = 1$$

This means that the states which are closer to the terminal state clearly get to the terminal state faster, hence their value goes up. 

Another way of solving this problem is to change the rewards themselves, so that at each time step, the reward is $-1$ and at the terminal state it's $0$. So this will "punish" the agent for staying long in the maze, just like with the case where we have discounts. In the following code you will see that this is indeed the case.

In [14]:
def maze(s):
  maze = -1*np.ones((s, s))
  maze[0,0], maze[-1,-1] = 0, 0
  return maze
print(maze(4))

def robot_experiment(iterations, discount, maze_size):
  ITERATIONS = iterations
  grid = maze(maze_size)
  gridCollection = [maze(4)]
  probs = np.array([0.25, 0.25, 0.25, 0.25])
  for iter in range(ITERATIONS):
      for row in range(grid.shape[0]):
        for col in range(grid.shape[1]):
          index = [row, col]
          rowID, colID = findCells(index, grid)
          if rowID == None or colID == None:
            grid[row, col] = grid[row, col]
          else:
            last_grid = gridCollection[iter]
            grid[row, col] = discount*last_grid[rowID, colID] @ probs
      new_grid = grid.copy()
      gridCollection.append(new_grid)
      if (iter+1)% iterations==0:
        print(f"Iteration: {iter+1}, grid:")
        print(grid)
        print("\n")

[[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1.  0.]]


In [15]:
robot_experiment(200, 0.1, 4)
robot_experiment(200, 0.9, 4)
robot_experiment(200, 1, 4)

Iteration: 200, grid:
[[ 0.00000000e+000 -1.29049831e-205 -1.91228610e-205 -2.13996124e-205]
 [-1.29049831e-205 -1.68461097e-205 -1.89949420e-205 -1.91228610e-205]
 [-1.91228610e-205 -1.89949420e-205 -1.68461097e-205 -1.29049831e-205]
 [-2.13996124e-205 -1.91228610e-205 -1.29049831e-205  0.00000000e+000]]


Iteration: 200, grid:
[[ 0.00000000e+00 -9.10456764e-15 -1.34913297e-14 -1.50975958e-14]
 [-9.10456764e-15 -1.18850636e-14 -1.34010818e-14 -1.34913297e-14]
 [-1.34913297e-14 -1.34010818e-14 -1.18850636e-14 -9.10456764e-15]
 [-1.50975958e-14 -1.34913297e-14 -9.10456764e-15  0.00000000e+00]]


Iteration: 200, grid:
[[ 0.00000000e+00 -1.29049831e-05 -1.91228610e-05 -2.13996124e-05]
 [-1.29049831e-05 -1.68461097e-05 -1.89949420e-05 -1.91228610e-05]
 [-1.91228610e-05 -1.89949420e-05 -1.68461097e-05 -1.29049831e-05]
 [-2.13996124e-05 -1.91228610e-05 -1.29049831e-05  0.00000000e+00]]


